In [1]:
# this file uses the biomart data and create the gtf file of: 
# [chr] [source=lilach] [feature=TSS] [start TSS] [end TSS] [score=1] [srand +-] [frame = '.' becuse TSS] [transcript name, gene name, gene type...]

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


#this script takes some edgeR table results and analize the expressed genes, try to understad which genes are differentialy expressed among the samples. 
#the analysis based on q_val
import sys
!{sys.executable} -m pip install sklearn

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.decomposition import PCA
import math
import re
import ast
import itertools
import scipy


In [3]:
# go on the biomart data and filtering the good transcripts

In [4]:
# old path: biomart=pd.read_csv(r'C:\Users\TzachiHNB1\Documents\docs_from_lilac\tzachi_hagai\stefan_bulk\promoter_analysis\homo\transcripts_info_biomart.txt', sep='\t')
biomart=pd.read_csv(r'transcripts_info_biomart.txt', sep='\t')
biomart

,Gene stable ID,Transcript stable ID,Strand,Transcription start site (TSS),Transcript support level (TSL),Transcript length (including UTRs and CDS),Gene type,Chromosome/scaffold name
0,ENSG00000210049,ENST00000387314,1,577,tslNA,71,Mt_tRNA,MT
1,ENSG00000211459,ENST00000389680,1,648,tslNA,954,Mt_rRNA,MT
2,ENSG00000210077,ENST00000387342,1,1602,tslNA,69,Mt_tRNA,MT
3,ENSG00000210082,ENST00000387347,1,1671,tslNA,1559,Mt_rRNA,MT
4,ENSG00000209082,ENST00000386347,1,3230,tslNA,75,Mt_tRNA,MT
...,...,...,...,...,...,...,...,...
249601,ENSG00000285363,ENST00000642800,1,129728744,NaN,1122,processed_pseudogene,CHR_HSCHR8_7_CTG7
249602,ENSG00000285114,ENST00000645112,-1,129761915,NaN,1177,protein_coding,CHR_HSCHR8_7_CTG7
249603,ENSG00000285114,ENST00000642712,-1,129795674,NaN,2450,protein_coding,CHR_HSCHR8_7_CTG7
249604,ENSG00000285114,ENST00000646090,-1,129786637,NaN,1831,protein_coding,CHR_HSCHR8_7_CTG7


In [5]:
biomart_protein_coding=biomart[biomart['Gene type']=='protein_coding']
biomart_protein_coding

,Gene stable ID,Transcript stable ID,Strand,Transcription start site (TSS),Transcript support level (TSL),Transcript length (including UTRs and CDS),Gene type,Chromosome/scaffold name
5,ENSG00000198888,ENST00000361390,1,3307,tslNA,956,protein_coding,MT
9,ENSG00000198763,ENST00000361453,1,4470,tslNA,1042,protein_coding,MT
15,ENSG00000198804,ENST00000361624,1,5904,tslNA,1542,protein_coding,MT
18,ENSG00000198712,ENST00000361739,1,7586,tslNA,684,protein_coding,MT
20,ENSG00000228253,ENST00000361851,1,8366,tslNA,207,protein_coding,MT
...,...,...,...,...,...,...,...,...
249593,ENSG00000284869,ENST00000646013,1,128343834,NaN,745,protein_coding,CHR_HSCHR3_9_CTG2_1
249602,ENSG00000285114,ENST00000645112,-1,129761915,NaN,1177,protein_coding,CHR_HSCHR8_7_CTG7
249603,ENSG00000285114,ENST00000642712,-1,129795674,NaN,2450,protein_coding,CHR_HSCHR8_7_CTG7
249604,ENSG00000285114,ENST00000646090,-1,129786637,NaN,1831,protein_coding,CHR_HSCHR8_7_CTG7


In [6]:
# remove secondary hap 

h=[i for i in biomart_protein_coding.index if 'CHR' not in biomart_protein_coding.at[i,'Chromosome/scaffold name']]

biomart_protein_coding_no_CHR=biomart_protein_coding.loc[h]
#biomart_protein_coding_no_CHR=biomart_protein_coding[(biomart_protein_coding['Chromosome/scaffold name']=='1') |
 #                                                   (biomart_protein_coding['Chromosome/scaffold name']=='2') |
  #                                                  (biomart_protein_coding['Chromosome/scaffold name']=='3') |
   #                                                 (biomart_protein_coding['Chromosome/scaffold name']=='4') |
    #                                                (biomart_protein_coding['Chromosome/scaffold name']=='5') |
     #                                               (biomart_protein_coding['Chromosome/scaffold name']=='6') |
      #                                              (biomart_protein_coding['Chromosome/scaffold name']=='7') |
       #                                             (biomart_protein_coding['Chromosome/scaffold name']=='8') |
        #                                            (biomart_protein_coding['Chromosome/scaffold name']=='9') |
         #                                           (biomart_protein_coding['Chromosome/scaffold name']=='10') |
          #                                          (biomart_protein_coding['Chromosome/scaffold name']=='11') |
           #                                         (biomart_protein_coding['Chromosome/scaffold name']=='12') |
            #                                        (biomart_protein_coding['Chromosome/scaffold name']=='13') |
             #                                       (biomart_protein_coding['Chromosome/scaffold name']=='14') |
              #                                      (biomart_protein_coding['Chromosome/scaffold name']=='15') |
               #                                     (biomart_protein_coding['Chromosome/scaffold name']=='16') |
                #                                    (biomart_protein_coding['Chromosome/scaffold name']=='17') |
                 #                                   (biomart_protein_coding['Chromosome/scaffold name']=='18') |
                  #                                  (biomart_protein_coding['Chromosome/scaffold name']=='19') |
                   #                                 (biomart_protein_coding['Chromosome/scaffold name']=='20') |
                    #                                 (biomart_protein_coding['Chromosome/scaffold name']=='22') |
                     #                                (biomart_protein_coding['Chromosome/scaffold name']=='22') |
                      #                              (biomart_protein_coding['Chromosome/scaffold name']=='X') |
                       #                             (biomart_protein_coding['Chromosome/scaffold name']=='Y') |
                        #                            (biomart_protein_coding['Chromosome/scaffold name']=='MT') ]
biomart_protein_coding_no_CHR











,Gene stable ID,Transcript stable ID,Strand,Transcription start site (TSS),Transcript support level (TSL),Transcript length (including UTRs and CDS),Gene type,Chromosome/scaffold name
5,ENSG00000198888,ENST00000361390,1,3307,tslNA,956,protein_coding,MT
9,ENSG00000198763,ENST00000361453,1,4470,tslNA,1042,protein_coding,MT
15,ENSG00000198804,ENST00000361624,1,5904,tslNA,1542,protein_coding,MT
18,ENSG00000198712,ENST00000361739,1,7586,tslNA,684,protein_coding,MT
20,ENSG00000228253,ENST00000361851,1,8366,tslNA,207,protein_coding,MT
...,...,...,...,...,...,...,...,...
243592,ENSG00000278704,ENST00000618686,-1,58376,tslNA,2237,protein_coding,GL000009.2
243599,ENSG00000278782,ENST00000619130,1,1157687,tsl5,285,protein_coding,KI270728.1
243654,ENSG00000286094,ENST00000382776,-1,766919,tsl5,1116,protein_coding,5
243659,ENSG00000286094,ENST00000622126,-1,751234,tsl5,1301,protein_coding,5


In [7]:
# check the chromosomes that left
np.unique(list(biomart_protein_coding_no_CHR['Chromosome/scaffold name']))

array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19',
       '2', '20', '21', '22', '3', '4', '5', '6', '7', '8', '9',
       'GL000009.2', 'GL000194.1', 'GL000195.1', 'GL000205.2',
       'GL000213.1', 'GL000218.1', 'GL000219.1', 'KI270711.1',
       'KI270713.1', 'KI270721.1', 'KI270726.1', 'KI270727.1',
       'KI270728.1', 'KI270731.1', 'KI270734.1', 'MT', 'X', 'Y'],
      dtype='<U10')

In [8]:
# replacing -1 > - and 1 > +
biomart_protein_coding_no_CHR.loc[(biomart_protein_coding_no_CHR.Strand == 1), 'Strand'] = '+'
biomart_protein_coding_no_CHR.loc[(biomart_protein_coding_no_CHR.Strand == -1), 'Strand'] = '-'

In [9]:
biomart_protein_coding_no_CHR

,Gene stable ID,Transcript stable ID,Strand,Transcription start site (TSS),Transcript support level (TSL),Transcript length (including UTRs and CDS),Gene type,Chromosome/scaffold name
5,ENSG00000198888,ENST00000361390,+,3307,tslNA,956,protein_coding,MT
9,ENSG00000198763,ENST00000361453,+,4470,tslNA,1042,protein_coding,MT
15,ENSG00000198804,ENST00000361624,+,5904,tslNA,1542,protein_coding,MT
18,ENSG00000198712,ENST00000361739,+,7586,tslNA,684,protein_coding,MT
20,ENSG00000228253,ENST00000361851,+,8366,tslNA,207,protein_coding,MT
...,...,...,...,...,...,...,...,...
243592,ENSG00000278704,ENST00000618686,-,58376,tslNA,2237,protein_coding,GL000009.2
243599,ENSG00000278782,ENST00000619130,+,1157687,tsl5,285,protein_coding,KI270728.1
243654,ENSG00000286094,ENST00000382776,-,766919,tsl5,1116,protein_coding,5
243659,ENSG00000286094,ENST00000622126,-,751234,tsl5,1301,protein_coding,5


In [10]:
len(np.unique(list(biomart_protein_coding_no_CHR['Gene stable ID'])))

19968

In [11]:
# filter the double transcripts- for each gene i create a mini df with its transcripts only and than
# take one transcripts that is with the lowest TLS and if all of them are the same- take the longest 

In [12]:
biomart_protein_coding_no_CHR[pd.isnull(biomart_protein_coding_no_CHR['Transcript support level (TSL)'])==True]

,Gene stable ID,Transcript stable ID,Strand,Transcription start site (TSS),Transcript support level (TSL),Transcript length (including UTRs and CDS),Gene type,Chromosome/scaffold name
143,ENSG00000134516,ENST00000648741,+,169637268,NaN,506,protein_coding,5
211,ENSG00000153879,ENST00000652630,+,33373685,NaN,2810,protein_coding,19
327,ENSG00000183150,ENST00000651487,-,12696183,NaN,1910,protein_coding,12
363,ENSG00000171561,ENST00000641541,-,75096876,NaN,8795,protein_coding,11
369,ENSG00000171561,ENST00000641593,-,75096876,NaN,8733,protein_coding,11
...,...,...,...,...,...,...,...,...
235738,ENSG00000284723,ENST00000641651,+,48525632,NaN,939,protein_coding,12
235771,ENSG00000183337,ENST00000673391,-,40096612,NaN,6196,protein_coding,X
235773,ENSG00000183337,ENST00000672265,-,40056233,NaN,2184,protein_coding,X
235776,ENSG00000183337,ENST00000672922,-,40097979,NaN,752,protein_coding,X


In [13]:
biomart_protein_coding_no_CHR[biomart_protein_coding_no_CHR['Gene stable ID']=='ENSG00000198888']

,Gene stable ID,Transcript stable ID,Strand,Transcription start site (TSS),Transcript support level (TSL),Transcript length (including UTRs and CDS),Gene type,Chromosome/scaffold name
5,ENSG00000198888,ENST00000361390,+,3307,tslNA,956,protein_coding,MT


In [14]:
# for convinence i will replace the TLS 
for i in biomart_protein_coding_no_CHR.index:
    print(i)
    TSL_item=biomart_protein_coding_no_CHR.at[i,'Transcript support level (TSL)']
    #print(type(TSL_item))
    
    if pd.isnull(TSL_item)==True:
        biomart_protein_coding_no_CHR.at[i,'Transcript support level (TSL)']=7
    else:
        if TSL_item.startswith('tsl1'):
            biomart_protein_coding_no_CHR.at[i,'Transcript support level (TSL)']=1
        elif TSL_item.startswith('tsl2'):
            biomart_protein_coding_no_CHR.at[i,'Transcript support level (TSL)']=2
        elif TSL_item.startswith('tsl3'):
            biomart_protein_coding_no_CHR.at[i,'Transcript support level (TSL)']=3
        elif TSL_item.startswith('tsl4'):
            biomart_protein_coding_no_CHR.at[i,'Transcript support level (TSL)']=4
        elif TSL_item.startswith('tsl5'):
            biomart_protein_coding_no_CHR.at[i,'Transcript support level (TSL)']=5
        elif TSL_item.startswith('tslNA'):
            biomart_protein_coding_no_CHR.at[i,'Transcript support level (TSL)']=6
        
    



5
9
15
18
20
21
22
24
26
27
31
32
34
57
75
77
83
86
91
93
100
103
109
119
124
125
127
131
132
138
143
145
146
151
152
159
161
162
185
186
190
191
192
193
194
201
202
207
209
210
211
222
224
225
238
240
242
248
249
253
257
259
269
272
294
295
296
297
298
299
302
304
308
309
311
319
320
321
322
325
326
327
328
337
339
340
341
342
343
354
355
357
358
361
362
363
365
369
370
372
373
374
378
379
381
382
387
388
389
390
392
394
395
396
400
401
403
404
408
410
413
414
415
416
418
420
421
422
424
426
436
438
439
441
442
450
451
452
455
456
458
462
463
465
468
471
473
474
476
478
480
481
486
488
490
492
493
494
497
498
501
502
503
504
505
506
509
511
512
513
514
517
518
519
520
522
525
528
530
531
532
534
537
539
540
542
544
547
548
554
555
556
558
559
561
562
563
564
567
573
578
580
581
582
584
585
587
588
590
591
595
597
598
599
600
602
603
604
607
608
609
610
611
612
613
615
616
617
618
621
622
623
624
625
626
627
629
632
633
636
642
644
646
648
649
650
652
654
656
660
661
662
665
666
668
67

3128
3130
3131
3133
3134
3136
3137
3138
3139
3140
3141
3142
3143
3145
3147
3149
3150
3152
3153
3154
3155
3156
3157
3158
3161
3162
3164
3165
3166
3167
3168
3173
3174
3175
3176
3177
3178
3179
3180
3181
3183
3184
3185
3188
3189
3190
3191
3192
3194
3195
3196
3197
3198
3199
3200
3201
3202
3203
3204
3205
3206
3207
3210
3211
3212
3213
3214
3215
3216
3218
3220
3221
3222
3223
3224
3225
3227
3228
3229
3231
3232
3233
3235
3236
3238
3239
3240
3241
3243
3245
3246
3247
3248
3249
3250
3251
3252
3256
3258
3260
3261
3262
3263
3265
3266
3267
3268
3269
3270
3272
3273
3274
3275
3277
3280
3281
3282
3283
3284
3288
3289
3290
3293
3294
3295
3297
3298
3299
3301
3303
3304
3305
3307
3308
3309
3310
3311
3313
3314
3316
3317
3318
3319
3321
3323
3324
3325
3326
3327
3328
3329
3330
3331
3333
3334
3337
3339
3341
3342
3343
3345
3346
3347
3348
3349
3350
3352
3353
3355
3357
3358
3359
3360
3361
3363
3364
3365
3366
3367
3368
3370
3371
3373
3374
3376
3377
3378
3379
3380
3382
3386
3387
3388
3390
3392
3393
3394
3395
3397
3398


6265
6266
6267
6268
6269
6270
6273
6275
6276
6278
6279
6282
6283
6284
6285
6286
6287
6288
6289
6290
6291
6292
6293
6294
6295
6296
6297
6298
6299
6303
6304
6306
6308
6310
6312
6313
6314
6316
6318
6319
6320
6321
6324
6325
6328
6330
6331
6332
6333
6338
6340
6342
6343
6344
6347
6348
6349
6350
6351
6353
6354
6355
6356
6357
6359
6360
6361
6362
6364
6365
6367
6368
6369
6370
6372
6373
6376
6384
6385
6386
6387
6389
6390
6391
6393
6394
6395
6396
6397
6398
6399
6400
6401
6402
6403
6405
6406
6407
6408
6409
6411
6412
6413
6414
6415
6416
6417
6419
6420
6421
6423
6424
6426
6427
6428
6429
6430
6431
6432
6433
6434
6435
6436
6437
6440
6441
6442
6443
6444
6446
6447
6448
6449
6451
6452
6453
6454
6455
6456
6457
6458
6459
6460
6461
6462
6463
6464
6465
6466
6467
6468
6469
6470
6471
6472
6474
6475
6476
6477
6478
6479
6480
6481
6483
6484
6485
6486
6487
6488
6489
6490
6491
6492
6493
6494
6495
6496
6497
6498
6499
6500
6501
6502
6503
6505
6506
6507
6508
6509
6510
6511
6512
6513
6514
6515
6516
6517
6518
6519
6520


9347
9350
9352
9356
9358
9361
9364
9366
9367
9368
9371
9372
9375
9376
9377
9378
9381
9382
9383
9385
9386
9388
9390
9392
9393
9395
9398
9400
9403
9405
9406
9407
9414
9415
9418
9419
9422
9423
9424
9426
9431
9432
9433
9435
9437
9438
9444
9448
9450
9451
9453
9455
9456
9458
9460
9465
9466
9467
9468
9473
9474
9475
9477
9478
9479
9481
9482
9483
9484
9485
9486
9487
9489
9490
9491
9495
9496
9497
9500
9501
9506
9507
9509
9511
9515
9516
9518
9521
9522
9523
9526
9527
9530
9531
9533
9534
9535
9536
9538
9539
9540
9544
9545
9546
9547
9551
9552
9553
9555
9557
9559
9560
9565
9568
9569
9570
9572
9574
9575
9576
9580
9581
9582
9585
9586
9590
9592
9593
9594
9595
9597
9598
9602
9603
9604
9605
9606
9610
9611
9612
9613
9615
9618
9619
9623
9624
9627
9632
9634
9637
9638
9642
9644
9645
9646
9651
9652
9653
9656
9661
9664
9666
9667
9669
9670
9671
9673
9677
9680
9681
9682
9687
9690
9693
9700
9701
9704
9705
9706
9708
9710
9713
9714
9717
9722
9724
9726
9727
9733
9734
9735
9737
9738
9741
9744
9746
9747
9749
9753
9755


12469
12471
12472
12473
12474
12475
12477
12478
12480
12482
12483
12484
12485
12486
12487
12488
12489
12490
12491
12492
12494
12495
12497
12499
12500
12505
12506
12507
12508
12510
12512
12513
12515
12516
12517
12519
12520
12521
12522
12523
12526
12527
12529
12530
12531
12532
12534
12535
12537
12538
12539
12540
12542
12543
12544
12545
12546
12548
12549
12551
12552
12553
12554
12555
12556
12557
12558
12559
12560
12561
12564
12565
12566
12567
12568
12569
12570
12571
12573
12574
12575
12576
12578
12579
12580
12581
12582
12583
12584
12585
12588
12589
12592
12593
12597
12598
12601
12603
12605
12606
12607
12608
12609
12611
12612
12613
12614
12615
12617
12618
12620
12623
12625
12628
12631
12632
12634
12635
12638
12641
12642
12643
12647
12648
12650
12654
12655
12657
12662
12663
12665
12668
12669
12671
12673
12675
12678
12679
12681
12683
12685
12689
12691
12695
12697
12699
12700
12701
12703
12708
12709
12714
12716
12717
12720
12721
12727
12729
12731
12735
12736
12738
12739
12740
12743
12745
1274

...

225798
225799
225800
225801
225803
225804
225805
225807
225808
225809
225810
225813
225819
225820
225821
225822
225824
225825
225826
225827
225828
225829
225830
225831
225832
225833
225834
225835
225836
225837
225838
225839
225840
225841
225842
225843
225844
225845
225846
225847
225848
225849
225850
225851
225852
225853
225854
225855
225856
225857
225858
225859
225860
225861
225862
225863
225864
225867
225870
225871
225872
225873
225874
225875
225876
225877
225878
225879
225880
225881
225882
225883
225884
225885
225886
225887
225888
225889
225890
225891
225892
225893
225894
225896
225897
225898
225900
225901
225904
225907
225908
225911
225912
225913
225914
225915
225916
225917
225918
225919
225920
225921
225922
225923
225925
225926
225927
225928
225929
225930
225931
225932
225934
225935
225936
225937
225938
225939
225940
225941
225943
225944
225945
225948
225949
225950
225951
225952
225953
225954
225955
225958
225959
225960
225961
225962
225963
225964
225965
225967
225969
225970
225972

228158
228159
228160
228161
228162
228163
228164
228165
228166
228167
228168
228169
228170
228171
228172
228173
228174
228175
228176
228177
228178
228179
228180
228181
228182
228183
228184
228185
228186
228187
228188
228189
228190
228191
228192
228193
228194
228195
228196
228197
228198
228199
228200
228201
228202
228203
228204
228205
228206
228207
228208
228209
228210
228211
228212
228213
228214
228215
228216
228217
228218
228219
228220
228221
228222
228223
228224
228225
228226
228227
228228
228229
228230
228231
228232
228233
228234
228236
228237
228240
228242
228244
228245
228248
228250
228256
228258
228261
228263
228267
228273
228275
228278
228280
228281
228282
228284
228286
228287
228289
228291
228292
228293
228295
228296
228299
228300
228303
228304
228305
228306
228307
228310
228311
228313
228315
228317
228318
228321
228324
228327
228329
228332
228335
228340
228341
228342
228343
228344
228345
228346
228347
228348
228349
228350
228351
228354
228355
228356
228359
228360
228362
228363

230708
230709
230710
230711
230712
230713
230714
230715
230716
230717
230719
230721
230722
230723
230724
230726
230727
230730
230731
230734
230735
230737
230738
230740
230742
230743
230745
230747
230748
230750
230752
230753
230755
230756
230758
230759
230760
230766
230769
230771
230773
230774
230776
230777
230779
230781
230782
230783
230788
230789
230790
230802
230804
230805
230807
230809
230818
230819
230820
230821
230822
230823
230824
230826
230827
230828
230829
230830
230831
230832
230833
230834
230835
230837
230839
230840
230841
230842
230843
230844
230845
230846
230847
230848
230849
230850
230851
230852
230853
230854
230855
230856
230858
230859
230860
230862
230863
230864
230865
230866
230867
230868
230869
230871
230872
230873
230874
230875
230876
230877
230878
230879
230880
230881
230882
230883
230884
230885
230886
230887
230888
230889
230890
230891
230892
230893
230894
230895
230896
230897
230898
230899
230900
230901
230902
230903
230904
230905
230906
230908
230909
230910
230911

233236
233237
233238
233239
233240
233241
233243
233245
233248
233249
233251
233255
233258
233260
233274
233276
233277
233278
233279
233280
233284
233285
233286
233287
233288
233289
233290
233291
233292
233293
233294
233295
233296
233297
233298
233299
233300
233301
233302
233303
233304
233305
233306
233307
233308
233309
233310
233311
233312
233313
233314
233315
233316
233317
233318
233319
233320
233321
233322
233323
233324
233325
233326
233327
233328
233329
233330
233331
233332
233333
233334
233335
233336
233337
233338
233339
233340
233341
233342
233343
233344
233345
233346
233355
233358
233359
233360
233361
233364
233365
233367
233369
233371
233373
233374
233376
233378
233379
233381
233382
233384
233405
233409
233412
233413
233414
233415
233416
233417
233419
233421
233422
233423
233424
233425
233426
233428
233430
233431
233433
233434
233435
233436
233437
233438
233439
233440
233441
233442
233443
233445
233446
233447
233448
233449
233455
233457
233460
233461
233463
233464
233466
233468

235707
235708
235709
235710
235711
235712
235713
235714
235715
235716
235717
235718
235719
235720
235721
235722
235723
235724
235725
235730
235731
235732
235733
235734
235735
235736
235737
235738
235739
235742
235743
235744
235745
235746
235747
235748
235749
235750
235751
235752
235753
235754
235756
235757
235758
235761
235762
235763
235764
235765
235766
235767
235768
235769
235770
235771
235772
235773
235774
235775
235776
235777
235778
235779
235780
235781
235782
235783
235784
235785
235786
235787
235788
235791
235792
235793
235801
236140
236778
236937
236946
236998
237010
237089
237646
237659
237736
237865
238118
238144
238179
238203
238227
238332
238602
238761
239374
239432
239596
240042
240063
240254
240294
240301
240314
240351
240551
240924
241676
242194
242346
242357
242497
243064
243086
243248
243261
243276
243592
243599
243654
243659
243687


In [15]:
biomart_protein_coding_no_CHR

,Gene stable ID,Transcript stable ID,Strand,Transcription start site (TSS),Transcript support level (TSL),Transcript length (including UTRs and CDS),Gene type,Chromosome/scaffold name
5,ENSG00000198888,ENST00000361390,+,3307,6,956,protein_coding,MT
9,ENSG00000198763,ENST00000361453,+,4470,6,1042,protein_coding,MT
15,ENSG00000198804,ENST00000361624,+,5904,6,1542,protein_coding,MT
18,ENSG00000198712,ENST00000361739,+,7586,6,684,protein_coding,MT
20,ENSG00000228253,ENST00000361851,+,8366,6,207,protein_coding,MT
...,...,...,...,...,...,...,...,...
243592,ENSG00000278704,ENST00000618686,-,58376,6,2237,protein_coding,GL000009.2
243599,ENSG00000278782,ENST00000619130,+,1157687,5,285,protein_coding,KI270728.1
243654,ENSG00000286094,ENST00000382776,-,766919,5,1116,protein_coding,5
243659,ENSG00000286094,ENST00000622126,-,751234,5,1301,protein_coding,5


In [16]:
biomart_protein_coding_no_CHR[biomart_protein_coding_no_CHR['Gene stable ID']=='ENSG00000198888']

,Gene stable ID,Transcript stable ID,Strand,Transcription start site (TSS),Transcript support level (TSL),Transcript length (including UTRs and CDS),Gene type,Chromosome/scaffold name
5,ENSG00000198888,ENST00000361390,+,3307,6,956,protein_coding,MT


In [17]:
np.unique(list(biomart_protein_coding_no_CHR['Transcript support level (TSL)']))


array([1, 2, 3, 4, 5, 6, 7])

In [18]:
transcripts_to_save=[]

for gene in np.unique(list(biomart_protein_coding_no_CHR['Gene stable ID'])):
    mini_df_for_gene=biomart_protein_coding_no_CHR[biomart_protein_coding_no_CHR['Gene stable ID']==gene]
    #print(mini_df_for_gene['Transcript support level (TSL)'])
    min_tsl=np.min(mini_df_for_gene['Transcript support level (TSL)'])
    #print(min_tsl)
    mini_df_for_gene_only_min=mini_df_for_gene[mini_df_for_gene['Transcript support level (TSL)']==min_tsl]
    #print(mini_df_for_gene_only_min)
    
    if len(mini_df_for_gene_only_min.index)==1:
        transcript_to_save=list(mini_df_for_gene_only_min['Transcript stable ID'])[0]
        transcripts_to_save.append(transcript_to_save)
        #print(transcript_to_save)
        #print('\n\n')
    elif len(mini_df_for_gene_only_min.index)>1:
        max_seq_len=np.max(mini_df_for_gene_only_min['Transcript length (including UTRs and CDS)'])
        mini_df_for_gene_only_min_and_maxlen=mini_df_for_gene_only_min[mini_df_for_gene_only_min['Transcript length (including UTRs and CDS)']==max_seq_len]
        
        transcript_to_save=list(mini_df_for_gene_only_min_and_maxlen['Transcript stable ID'])[0]
        transcripts_to_save.append(transcript_to_save)
    print(len(transcripts_to_save))
    
    
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064


3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706


5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353


6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988
6989
6990
6991
6992
6993
6994
6995
6996
6997
6998
6999
7000
7001
7002
7003


...

14239
14240
14241
14242
14243
14244
14245
14246
14247
14248
14249
14250
14251
14252
14253
14254
14255
14256
14257
14258
14259
14260
14261
14262
14263
14264
14265
14266
14267
14268
14269
14270
14271
14272
14273
14274
14275
14276
14277
14278
14279
14280
14281
14282
14283
14284
14285
14286
14287
14288
14289
14290
14291
14292
14293
14294
14295
14296
14297
14298
14299
14300
14301
14302
14303
14304
14305
14306
14307
14308
14309
14310
14311
14312
14313
14314
14315
14316
14317
14318
14319
14320
14321
14322
14323
14324
14325
14326
14327
14328
14329
14330
14331
14332
14333
14334
14335
14336
14337
14338
14339
14340
14341
14342
14343
14344
14345
14346
14347
14348
14349
14350
14351
14352
14353
14354
14355
14356
14357
14358
14359
14360
14361
14362
14363
14364
14365
14366
14367
14368
14369
14370
14371
14372
14373
14374
14375
14376
14377
14378
14379
14380
14381
14382
14383
14384
14385
14386
14387
14388
14389
14390
14391
14392
14393
14394
14395
14396
14397
14398
14399
14400
14401
14402
14403
14404
1440

15606
15607
15608
15609
15610
15611
15612
15613
15614
15615
15616
15617
15618
15619
15620
15621
15622
15623
15624
15625
15626
15627
15628
15629
15630
15631
15632
15633
15634
15635
15636
15637
15638
15639
15640
15641
15642
15643
15644
15645
15646
15647
15648
15649
15650
15651
15652
15653
15654
15655
15656
15657
15658
15659
15660
15661
15662
15663
15664
15665
15666
15667
15668
15669
15670
15671
15672
15673
15674
15675
15676
15677
15678
15679
15680
15681
15682
15683
15684
15685
15686
15687
15688
15689
15690
15691
15692
15693
15694
15695
15696
15697
15698
15699
15700
15701
15702
15703
15704
15705
15706
15707
15708
15709
15710
15711
15712
15713
15714
15715
15716
15717
15718
15719
15720
15721
15722
15723
15724
15725
15726
15727
15728
15729
15730
15731
15732
15733
15734
15735
15736
15737
15738
15739
15740
15741
15742
15743
15744
15745
15746
15747
15748
15749
15750
15751
15752
15753
15754
15755
15756
15757
15758
15759
15760
15761
15762
15763
15764
15765
15766
15767
15768
15769
15770
15771
1577

16974
16975
16976
16977
16978
16979
16980
16981
16982
16983
16984
16985
16986
16987
16988
16989
16990
16991
16992
16993
16994
16995
16996
16997
16998
16999
17000
17001
17002
17003
17004
17005
17006
17007
17008
17009
17010
17011
17012
17013
17014
17015
17016
17017
17018
17019
17020
17021
17022
17023
17024
17025
17026
17027
17028
17029
17030
17031
17032
17033
17034
17035
17036
17037
17038
17039
17040
17041
17042
17043
17044
17045
17046
17047
17048
17049
17050
17051
17052
17053
17054
17055
17056
17057
17058
17059
17060
17061
17062
17063
17064
17065
17066
17067
17068
17069
17070
17071
17072
17073
17074
17075
17076
17077
17078
17079
17080
17081
17082
17083
17084
17085
17086
17087
17088
17089
17090
17091
17092
17093
17094
17095
17096
17097
17098
17099
17100
17101
17102
17103
17104
17105
17106
17107
17108
17109
17110
17111
17112
17113
17114
17115
17116
17117
17118
17119
17120
17121
17122
17123
17124
17125
17126
17127
17128
17129
17130
17131
17132
17133
17134
17135
17136
17137
17138
17139
1714

18360
18361
18362
18363
18364
18365
18366
18367
18368
18369
18370
18371
18372
18373
18374
18375
18376
18377
18378
18379
18380
18381
18382
18383
18384
18385
18386
18387
18388
18389
18390
18391
18392
18393
18394
18395
18396
18397
18398
18399
18400
18401
18402
18403
18404
18405
18406
18407
18408
18409
18410
18411
18412
18413
18414
18415
18416
18417
18418
18419
18420
18421
18422
18423
18424
18425
18426
18427
18428
18429
18430
18431
18432
18433
18434
18435
18436
18437
18438
18439
18440
18441
18442
18443
18444
18445
18446
18447
18448
18449
18450
18451
18452
18453
18454
18455
18456
18457
18458
18459
18460
18461
18462
18463
18464
18465
18466
18467
18468
18469
18470
18471
18472
18473
18474
18475
18476
18477
18478
18479
18480
18481
18482
18483
18484
18485
18486
18487
18488
18489
18490
18491
18492
18493
18494
18495
18496
18497
18498
18499
18500
18501
18502
18503
18504
18505
18506
18507
18508
18509
18510
18511
18512
18513
18514
18515
18516
18517
18518
18519
18520
18521
18522
18523
18524
18525
1852

19750
19751
19752
19753
19754
19755
19756
19757
19758
19759
19760
19761
19762
19763
19764
19765
19766
19767
19768
19769
19770
19771
19772
19773
19774
19775
19776
19777
19778
19779
19780
19781
19782
19783
19784
19785
19786
19787
19788
19789
19790
19791
19792
19793
19794
19795
19796
19797
19798
19799
19800
19801
19802
19803
19804
19805
19806
19807
19808
19809
19810
19811
19812
19813
19814
19815
19816
19817
19818
19819
19820
19821
19822
19823
19824
19825
19826
19827
19828
19829
19830
19831
19832
19833
19834
19835
19836
19837
19838
19839
19840
19841
19842
19843
19844
19845
19846
19847
19848
19849
19850
19851
19852
19853
19854
19855
19856
19857
19858
19859
19860
19861
19862
19863
19864
19865
19866
19867
19868
19869
19870
19871
19872
19873
19874
19875
19876
19877
19878
19879
19880
19881
19882
19883
19884
19885
19886
19887
19888
19889
19890
19891
19892
19893
19894
19895
19896
19897
19898
19899
19900
19901
19902
19903
19904
19905
19906
19907
19908
19909
19910
19911
19912
19913
19914
19915
1991

In [19]:
len(transcripts_to_save)


19968

In [20]:
#rows to keep according to transcripts to keep: 
rows_to_keep=[i for i in biomart_protein_coding_no_CHR.index if biomart_protein_coding_no_CHR.at[i,'Transcript stable ID'] in transcripts_to_save]


In [21]:
len(rows_to_keep)

19968

In [22]:
# leaving only the rows of relevant transcripts 
biomart_protein_coding_no_CHR=biomart_protein_coding_no_CHR.loc[rows_to_keep]
biomart_protein_coding_no_CHR

,Gene stable ID,Transcript stable ID,Strand,Transcription start site (TSS),Transcript support level (TSL),Transcript length (including UTRs and CDS),Gene type,Chromosome/scaffold name
5,ENSG00000198888,ENST00000361390,+,3307,6,956,protein_coding,MT
9,ENSG00000198763,ENST00000361453,+,4470,6,1042,protein_coding,MT
15,ENSG00000198804,ENST00000361624,+,5904,6,1542,protein_coding,MT
18,ENSG00000198712,ENST00000361739,+,7586,6,684,protein_coding,MT
20,ENSG00000228253,ENST00000361851,+,8366,6,207,protein_coding,MT
...,...,...,...,...,...,...,...,...
243276,ENSG00000204805,ENST00000605259,+,61861625,6,1401,protein_coding,9
243592,ENSG00000278704,ENST00000618686,-,58376,6,2237,protein_coding,GL000009.2
243599,ENSG00000278782,ENST00000619130,+,1157687,5,285,protein_coding,KI270728.1
243659,ENSG00000286094,ENST00000622126,-,751234,5,1301,protein_coding,5


In [23]:
# here we will leave only the relevant columns

In [24]:
new_df=biomart_protein_coding_no_CHR[['Chromosome/scaffold name','Transcription start site (TSS)','Strand','Gene stable ID','Transcript stable ID','Gene type']]
new_df

,Chromosome/scaffold name,Transcription start site (TSS),Strand,Gene stable ID,Transcript stable ID,Gene type
5,MT,3307,+,ENSG00000198888,ENST00000361390,protein_coding
9,MT,4470,+,ENSG00000198763,ENST00000361453,protein_coding
15,MT,5904,+,ENSG00000198804,ENST00000361624,protein_coding
18,MT,7586,+,ENSG00000198712,ENST00000361739,protein_coding
20,MT,8366,+,ENSG00000228253,ENST00000361851,protein_coding
...,...,...,...,...,...,...
243276,9,61861625,+,ENSG00000204805,ENST00000605259,protein_coding
243592,GL000009.2,58376,-,ENSG00000278704,ENST00000618686,protein_coding
243599,KI270728.1,1157687,+,ENSG00000278782,ENST00000619130,protein_coding
243659,5,751234,-,ENSG00000286094,ENST00000622126,protein_coding


In [25]:
for i in new_df.index:
    decription= 'Gene ID: {}, Transcript ID: {}, type: {}'.format(new_df.at[i,'Gene stable ID'],new_df.at[i,'Transcript stable ID'],new_df.at[i,'Gene type'])
    print(i)
    new_df.at[i,'description']=decription

5
9
15
18
20
21
22
24
26
27
31
32
34
57
75
86
93
109
125
127
131
146
151
152
190
192
193
194
202
257
272
294
297
298
328
342
362
389
392
403
404
418
439
450
498
505
522
528
534
578
580
598
602
607
609
611
613
618
636
650
666
668
687
697
705
707
734
736
783
795
799
803
817
820
823
828
841
851
861
865
873
874
875
892
894
895
896
914
943
977
988
990
1031
1040
1046
1047
1053
1055
1057
1062
1065
1066
1072
1074
1087
1102
1133
1141
1159
1168
1171
1185
1189
1203
1235
1236
1245
1251
1277
1285
1290
1294
1298
1307
1314
1315
1328
1346
1347
1349
1352
1359
1362
1363
1374
1386
1392
1395
1397
1400
1402
1411
1413
1421
1431
1432
1449
1479
1483
1504
1506
1510
1512
1534
1535
1546
1567
1579
1592
1613
1614
1619
1645
1651
1653
1685
1723
1732
1733
1747
1754
1755
1778
1782
1803
1806
1812
1828
1847
1864
1865
1868
1873
1893
1905
1930
1931
1935
1939
1979
1988
1999
2004
2005
2047
2053
2059
2064
2067
2090
2096
2113
2119
2122
2127
2130
2145
2151
2174
2186
2194
2218
2228
2250
2272
2306
2309
2313
2315
2330
2381
2395
2

C:\Users\TzachiHNB2\anaconda3\lib\site-packages\pandas\core\indexing.py:1598: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\TzachiHNB2\anaconda3\lib\site-packages\pandas\core\indexing.py:1719: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


18753
18760
18762
18765
18778
18781
18800
18807
18815
18816
18819
18828
18829
18837
18838
18846
18850
18854
18868
18882
18884
18885
18904
18941
18951
18971
18990
18998
19007
19009
19026
19036
19043
19053
19058
19067
19076
19085
19103
19107
19109
19123
19138
19165
19189
19194
19204
19215
19225
19238
19251
19264
19265
19282
19286
19294
19323
19327
19364
19391
19395
19476
19484
19485
19488
19492
19520
19524
19532
19544
19561
19571
19573
19651
19653
19677
19680
19681
19697
19711
19720
19721
19792
19793
19804
19811
19812
19832
19840
19841
19857
19865
19866
19877
19884
19888
19926
19953
19955
19972
19991
19999
20016
20037
20040
20062
20106
20113
20140
20152
20155
20199
20204
20207
20212
20238
20241
20245
20258
20271
20277
20323
20330
20340
20378
20381
20392
20421
20425
20430
20436
20437
20443
20461
20536
20573
20582
20590
20592
20594
20602
20603
20637
20656
20659
20675
20747
20751
20770
20778
20781
20782
20786
20787
20789
20832
20833
20836
20845
20861
20864
20874
20875
20894
20897
20904
2091

46177
46203
46215
46234
46257
46265
46267
46287
46309
46315
46345
46360
46377
46386
46393
46398
46400
46408
46463
46476
46478
46520
46530
46533
46569
46572
46600
46604
46626
46737
46757
46759
46772
46778
46793
46799
46807
46814
46820
46824
46848
46856
46866
46876
46881
46910
46922
46949
46952
46958
46972
46977
47003
47020
47033
47058
47075
47076
47099
47101
47112
47139
47141
47155
47157
47158
47160
47201
47204
47220
47238
47240
47245
47277
47295
47303
47308
47315
47316
47335
47380
47382
47397
47418
47442
47444
47489
47493
47494
47510
47523
47559
47568
47599
47608
47666
47675
47702
47713
47716
47722
47738
47741
47742
47792
47795
47810
47837
47850
47860
47869
47883
47910
47928
47934
47946
47947
47949
47954
47968
47970
47974
47980
47983
47984
48009
48028
48034
48037
48066
48090
48098
48104
48107
48118
48131
48183
48192
48231
48239
48244
48250
48256
48265
48277
48286
48319
48328
48330
48349
48363
48364
48372
48380
48393
48396
48458
48465
48470
48474
48482
48514
48530
48542
48558
48559
4856

67921
67935
67936
67948
67954
67969
67980
68014
68017
68018
68029
68037
68050
68076
68085
68130
68142
68144
68163
68174
68177
68222
68229
68233
68243
68245
68247
68255
68290
68311
68328
68337
68350
68369
68379
68407
68410
68422
68433
68434
68440
68450
68459
68462
68464
68498
68520
68535
68551
68561
68574
68579
68600
68646
68676
68692
68694
68697
68715
68732
68739
68740
68743
68749
68766
68771
68773
68789
68809
68833
68842
68845
68850
68859
68871
68875
68876
68878
68881
68896
68905
68908
68918
68936
68939
68951
68964
68991
69041
69045
69046
69053
69064
69084
69098
69103
69108
69118
69123
69135
69141
69145
69146
69147
69151
69163
69166
69174
69175
69191
69194
69212
69226
69233
69241
69251
69253
69257
69262
69266
69270
69272
69276
69278
69320
69330
69333
69368
69372
69388
69389
69426
69427
69438
69440
69466
69472
69483
69486
69491
69513
69563
69565
69574
69597
69604
69609
69615
69617
69633
69642
69643
69648
69650
69669
69674
69675
69687
69690
69719
69722
69735
69741
69743
69760
69780
6978

...

136668
136682
136684
136685
136699
136705
136707
136719
136722
136743
136748
136801
136805
136812
136814
136833
136834
136842
136845
136862
136887
136889
136913
136929
136937
136945
136947
136954
136960
136968
136975
136982
137000
137002
137011
137018
137021
137025
137054
137061
137062
137065
137066
137071
137075
137092
137119
137124
137134
137144
137146
137165
137170
137188
137213
137218
137219
137221
137233
137243
137249
137250
137273
137288
137294
137296
137300
137311
137340
137344
137365
137366
137380
137405
137407
137409
137416
137426
137434
137449
137468
137480
137485
137486
137489
137495
137504
137516
137555
137564
137572
137586
137595
137601
137602
137622
137629
137635
137640
137657
137667
137687
137697
137707
137717
137730
137737
137740
137741
137768
137781
137791
137817
137832
137838
137854
137870
137873
137884
137885
137889
137902
137910
137923
137959
137966
137971
137974
137985
137996
137999
138010
138029
138030
138050
138077
138099
138101
138133
138158
138183
138202
138203

159983
160013
160028
160042
160060
160076
160090
160109
160128
160145
160168
160188
160191
160194
160216
160231
160236
160241
160245
160249
160250
160256
160313
160361
160364
160389
160400
160402
160405
160428
160437
160451
160453
160460
160477
160503
160506
160509
160528
160538
160548
160553
160556
160578
160582
160583
160592
160594
160625
160628
160649
160660
160663
160717
160730
160735
160768
160774
160787
160793
160802
160818
160823
160826
160832
160847
160868
160869
160876
160893
160908
160921
160935
160944
160969
160988
160991
160999
161005
161019
161025
161046
161047
161080
161086
161089
161111
161116
161120
161136
161144
161145
161146
161150
161158
161164
161189
161194
161208
161209
161210
161224
161254
161255
161269
161275
161356
161357
161371
161377
161378
161381
161385
161425
161442
161443
161446
161448
161454
161459
161464
161467
161475
161476
161523
161542
161557
161558
161561
161562
161572
161583
161619
161630
161632
161657
161661
161680
161713
161735
161743
161749
161762

179433
179447
179466
179491
179502
179504
179505
179508
179517
179526
179554
179568
179569
179575
179596
179602
179616
179618
179635
179637
179641
179644
179651
179667
179668
179672
179681
179689
179696
179703
179710
179712
179721
179723
179736
179737
179761
179766
179769
179847
179849
179871
179893
179908
179909
179915
179918
179927
179929
179950
179953
179963
179964
179967
179978
179994
179995
180004
180012
180021
180048
180066
180074
180078
180093
180119
180138
180139
180179
180198
180199
180211
180218
180227
180239
180242
180243
180244
180248
180251
180259
180263
180299
180303
180311
180323
180325
180326
180329
180330
180352
180368
180404
180414
180419
180421
180433
180434
180441
180452
180483
180500
180527
180529
180534
180535
180536
180540
180542
180544
180562
180584
180586
180613
180617
180627
180634
180639
180643
180649
180650
180673
180676
180723
180729
180735
180754
180792
180818
180822
180826
180828
180856
180874
180875
180878
180903
180904
180906
180915
180921
180929
180942

206153
206159
206164
206169
206198
206220
206231
206235
206246
206260
206273
206295
206316
206328
206329
206334
206344
206345
206348
206361
206365
206369
206440
206443
206458
206467
206474
206475
206483
206490
206528
206536
206570
206581
206582
206589
206597
206601
206604
206618
206637
206652
206665
206711
206721
206728
206734
206741
206742
206809
206828
206853
206943
206951
206964
206967
206968
206992
207003
207025
207032
207047
207060
207088
207108
207146
207147
207148
207156
207159
207165
207169
207186
207192
207197
207222
207225
207230
207233
207239
207246
207268
207284
207290
207323
207325
207335
207349
207353
207356
207369
207387
207389
207395
207402
207417
207418
207432
207460
207462
207469
207471
207479
207512
207566
207572
207575
207577
207581
207588
207589
207606
207612
207616
207627
207636
207644
207669
207674
207682
207707
207726
207740
207745
207752
207762
207765
207768
207771
207775
207805
207815
207816
207821
207823
207832
207854
207855
207878
207890
207891
207899
207901

230007
230012
230013
230025
230045
230052
230067
230072
230077
230080
230116
230122
230131
230137
230149
230168
230169
230173
230177
230193
230210
230211
230237
230258
230289
230301
230323
230334
230358
230359
230410
230413
230425
230428
230431
230436
230446
230458
230472
230478
230487
230488
230491
230497
230520
230522
230563
230572
230577
230595
230617
230625
230631
230633
230637
230643
230655
230657
230677
230698
230717
230719
230731
230758
230759
230766
230769
230782
230790
230802
230821
230827
230828
230833
230837
230844
230853
230856
230886
230912
230923
230938
230950
230956
230965
230974
230995
231003
231013
231021
231034
231038
231061
231074
231083
231086
231091
231111
231166
231177
231182
231202
231206
231209
231224
231243
231258
231259
231280
231291
231295
231297
231317
231326
231342
231350
231364
231365
231372
231390
231396
231397
231399
231435
231438
231439
231480
231485
231490
231494
231496
231502
231507
231513
231525
231528
231537
231540
231558
231587
231598
231599
231606

In [26]:
new_df

,Chromosome/scaffold name,Transcription start site (TSS),Strand,Gene stable ID,Transcript stable ID,Gene type,description
5,MT,3307,+,ENSG00000198888,ENST00000361390,protein_coding,"Gene ID: ENSG00000198888, Transcript ID: ENST0..."
9,MT,4470,+,ENSG00000198763,ENST00000361453,protein_coding,"Gene ID: ENSG00000198763, Transcript ID: ENST0..."
15,MT,5904,+,ENSG00000198804,ENST00000361624,protein_coding,"Gene ID: ENSG00000198804, Transcript ID: ENST0..."
18,MT,7586,+,ENSG00000198712,ENST00000361739,protein_coding,"Gene ID: ENSG00000198712, Transcript ID: ENST0..."
20,MT,8366,+,ENSG00000228253,ENST00000361851,protein_coding,"Gene ID: ENSG00000228253, Transcript ID: ENST0..."
...,...,...,...,...,...,...,...
243276,9,61861625,+,ENSG00000204805,ENST00000605259,protein_coding,"Gene ID: ENSG00000204805, Transcript ID: ENST0..."
243592,GL000009.2,58376,-,ENSG00000278704,ENST00000618686,protein_coding,"Gene ID: ENSG00000278704, Transcript ID: ENST0..."
243599,KI270728.1,1157687,+,ENSG00000278782,ENST00000619130,protein_coding,"Gene ID: ENSG00000278782, Transcript ID: ENST0..."
243659,5,751234,-,ENSG00000286094,ENST00000622126,protein_coding,"Gene ID: ENSG00000286094, Transcript ID: ENST0..."


In [27]:
# adding source col, featue col, score, frame

new_df['source']='lilach'
new_df['feature']='TSS'
new_df['score']=1
new_df['frame']='.'


In [28]:
new_df.rename(columns={'Transcription start site (TSS)':'TSS start'},inplace=True)

In [29]:
new_df

,Chromosome/scaffold name,TSS start,Strand,Gene stable ID,Transcript stable ID,Gene type,description,source,feature,score,frame
5,MT,3307,+,ENSG00000198888,ENST00000361390,protein_coding,"Gene ID: ENSG00000198888, Transcript ID: ENST0...",lilach,TSS,1,.
9,MT,4470,+,ENSG00000198763,ENST00000361453,protein_coding,"Gene ID: ENSG00000198763, Transcript ID: ENST0...",lilach,TSS,1,.
15,MT,5904,+,ENSG00000198804,ENST00000361624,protein_coding,"Gene ID: ENSG00000198804, Transcript ID: ENST0...",lilach,TSS,1,.
18,MT,7586,+,ENSG00000198712,ENST00000361739,protein_coding,"Gene ID: ENSG00000198712, Transcript ID: ENST0...",lilach,TSS,1,.
20,MT,8366,+,ENSG00000228253,ENST00000361851,protein_coding,"Gene ID: ENSG00000228253, Transcript ID: ENST0...",lilach,TSS,1,.
...,...,...,...,...,...,...,...,...,...,...,...
243276,9,61861625,+,ENSG00000204805,ENST00000605259,protein_coding,"Gene ID: ENSG00000204805, Transcript ID: ENST0...",lilach,TSS,1,.
243592,GL000009.2,58376,-,ENSG00000278704,ENST00000618686,protein_coding,"Gene ID: ENSG00000278704, Transcript ID: ENST0...",lilach,TSS,1,.
243599,KI270728.1,1157687,+,ENSG00000278782,ENST00000619130,protein_coding,"Gene ID: ENSG00000278782, Transcript ID: ENST0...",lilach,TSS,1,.
243659,5,751234,-,ENSG00000286094,ENST00000622126,protein_coding,"Gene ID: ENSG00000286094, Transcript ID: ENST0...",lilach,TSS,1,.


In [30]:
new_df_with_TSS_end=new_df.copy()
new_df_with_TSS_end


,Chromosome/scaffold name,TSS start,Strand,Gene stable ID,Transcript stable ID,Gene type,description,source,feature,score,frame
5,MT,3307,+,ENSG00000198888,ENST00000361390,protein_coding,"Gene ID: ENSG00000198888, Transcript ID: ENST0...",lilach,TSS,1,.
9,MT,4470,+,ENSG00000198763,ENST00000361453,protein_coding,"Gene ID: ENSG00000198763, Transcript ID: ENST0...",lilach,TSS,1,.
15,MT,5904,+,ENSG00000198804,ENST00000361624,protein_coding,"Gene ID: ENSG00000198804, Transcript ID: ENST0...",lilach,TSS,1,.
18,MT,7586,+,ENSG00000198712,ENST00000361739,protein_coding,"Gene ID: ENSG00000198712, Transcript ID: ENST0...",lilach,TSS,1,.
20,MT,8366,+,ENSG00000228253,ENST00000361851,protein_coding,"Gene ID: ENSG00000228253, Transcript ID: ENST0...",lilach,TSS,1,.
...,...,...,...,...,...,...,...,...,...,...,...
243276,9,61861625,+,ENSG00000204805,ENST00000605259,protein_coding,"Gene ID: ENSG00000204805, Transcript ID: ENST0...",lilach,TSS,1,.
243592,GL000009.2,58376,-,ENSG00000278704,ENST00000618686,protein_coding,"Gene ID: ENSG00000278704, Transcript ID: ENST0...",lilach,TSS,1,.
243599,KI270728.1,1157687,+,ENSG00000278782,ENST00000619130,protein_coding,"Gene ID: ENSG00000278782, Transcript ID: ENST0...",lilach,TSS,1,.
243659,5,751234,-,ENSG00000286094,ENST00000622126,protein_coding,"Gene ID: ENSG00000286094, Transcript ID: ENST0...",lilach,TSS,1,.


In [31]:
# adding the TSS end according to the - or + strand

for i in new_df_with_TSS_end.index:
    print(i)
    strand= new_df_with_TSS_end.at[i,'Strand']
    TSS_start=new_df_with_TSS_end.at[i,'TSS start']
    
    if strand=='+':
        TSS_end_pre=TSS_start+1
        TSS_end=TSS_end_pre
        
        new_df_with_TSS_end.at[i,'TSS end']=TSS_end
        
        
    if strand=='-':
        TSS_end_pre=TSS_start-1
        TSS_end=TSS_start
        TSS_start=TSS_end_pre
        
        
        new_df_with_TSS_end.at[i,'TSS start']=TSS_start
        new_df_with_TSS_end.at[i,'TSS end']=TSS_end
        

5
9
15
18
20
21
22
24
26
27
31
32
34
57
75
86
93
109
125
127
131
146
151
152
190
192
193
194
202
257
272
294
297
298
328
342
362
389
392
403
404
418
439
450
498
505
522
528
534
578
580
598
602
607
609
611
613
618
636
650
666
668
687
697
705
707
734
736
783
795
799
803
817
820
823
828
841
851
861
865
873
874
875
892
894
895
896
914
943
977
988
990
1031
1040
1046
1047
1053
1055
1057
1062
1065
1066
1072
1074
1087
1102
1133
1141
1159
1168
1171
1185
1189
1203
1235
1236
1245
1251
1277
1285
1290
1294
1298
1307
1314
1315
1328
1346
1347
1349
1352
1359
1362
1363
1374
1386
1392
1395
1397
1400
1402
1411
1413
1421
1431
1432
1449
1479
1483
1504
1506
1510
1512
1534
1535
1546
1567
1579
1592
1613
1614
1619
1645
1651
1653
1685
1723
1732
1733
1747
1754
1755
1778
1782
1803
1806
1812
1828
1847
1864
1865
1868
1873
1893
1905
1930
1931
1935
1939
1979
1988
1999
2004
2005
2047
2053
2059
2064
2067
2090
2096
2113
2119
2122
2127
2130
2145
2151
2174
2186
2194
2218
2228
2250
2272
2306
2309
2313
2315
2330
2381
2395
2

23677
23684
23703
23716
23728
23741
23759
23771
23779
23783
23796
23811
23820
23827
23842
23848
23851
23861
23875
23881
23883
23892
23900
23901
23922
23933
23946
23948
23960
23973
24015
24021
24032
24037
24043
24046
24052
24057
24068
24079
24094
24097
24112
24118
24151
24154
24161
24183
24216
24234
24258
24264
24281
24284
24285
24298
24302
24306
24339
24353
24365
24368
24371
24379
24385
24403
24406
24415
24417
24435
24440
24469
24471
24474
24501
24510
24513
24518
24529
24534
24537
24540
24544
24570
24577
24581
24616
24624
24666
24675
24681
24683
24684
24691
24706
24707
24708
24723
24738
24741
24771
24803
24808
24830
24839
24850
24856
24858
24860
24865
24869
24870
24881
24898
24899
24913
24925
24928
24954
24967
24974
24984
24989
25025
25033
25036
25043
25071
25101
25110
25127
25130
25131
25135
25136
25137
25142
25148
25178
25186
25190
25191
25194
25248
25253
25259
25309
25314
25321
25333
25342
25353
25361
25392
25424
25457
25472
25479
25501
25514
25562
25563
25600
25602
25620
25632
2563

46910
46922
46949
46952
46958
46972
46977
47003
47020
47033
47058
47075
47076
47099
47101
47112
47139
47141
47155
47157
47158
47160
47201
47204
47220
47238
47240
47245
47277
47295
47303
47308
47315
47316
47335
47380
47382
47397
47418
47442
47444
47489
47493
47494
47510
47523
47559
47568
47599
47608
47666
47675
47702
47713
47716
47722
47738
47741
47742
47792
47795
47810
47837
47850
47860
47869
47883
47910
47928
47934
47946
47947
47949
47954
47968
47970
47974
47980
47983
47984
48009
48028
48034
48037
48066
48090
48098
48104
48107
48118
48131
48183
48192
48231
48239
48244
48250
48256
48265
48277
48286
48319
48328
48330
48349
48363
48364
48372
48380
48393
48396
48458
48465
48470
48474
48482
48514
48530
48542
48558
48559
48564
48591
48604
48609
48611
48616
48620
48630
48645
48655
48658
48665
48670
48680
48681
48686
48707
48712
48720
48734
48736
48740
48748
48771
48787
48790
48793
48797
48809
48814
48815
48824
48854
48868
48898
48917
48920
48921
48928
48955
48972
48981
48992
48994
49016
4904

68789
68809
68833
68842
68845
68850
68859
68871
68875
68876
68878
68881
68896
68905
68908
68918
68936
68939
68951
68964
68991
69041
69045
69046
69053
69064
69084
69098
69103
69108
69118
69123
69135
69141
69145
69146
69147
69151
69163
69166
69174
69175
69191
69194
69212
69226
69233
69241
69251
69253
69257
69262
69266
69270
69272
69276
69278
69320
69330
69333
69368
69372
69388
69389
69426
69427
69438
69440
69466
69472
69483
69486
69491
69513
69563
69565
69574
69597
69604
69609
69615
69617
69633
69642
69643
69648
69650
69669
69674
69675
69687
69690
69719
69722
69735
69741
69743
69760
69780
69784
69806
69810
69828
69834
69846
69870
69881
69887
69897
69930
69933
69935
69941
69949
69955
69957
69982
69989
69996
70004
70020
70050
70073
70095
70104
70108
70121
70125
70145
70157
70158
70168
70170
70188
70217
70219
70220
70245
70271
70276
70277
70282
70288
70301
70326
70336
70339
70340
70349
70362
70375
70431
70444
70445
70453
70457
70480
70484
70505
70516
70529
70543
70549
70562
70565
70566
7056

91775
91806
91856
91863
91880
91881
91904
91912
91916
91925
91942
91994
91996
92006
92009
92023
92030
92043
92047
92055
92058
92090
92098
92100
92103
92110
92115
92123
92132
92139
92150
92155
92158
92172
92195
92204
92214
92229
92232
92245
92260
92268
92279
92313
92317
92379
92385
92386
92401
92405
92410
92420
92445
92450
92458
92477
92480
92482
92489
92491
92493
92539
92544
92545
92570
92576
92583
92607
92618
92634
92666
92696
92697
92704
92709
92765
92780
92784
92792
92800
92820
92827
92843
92858
92861
92867
92876
92882
92892
92896
92904
92915
92927
92928
92944
92945
92967
92972
92984
92990
93006
93035
93043
93047
93078
93093
93128
93131
93137
93139
93141
93158
93170
93177
93193
93201
93203
93215
93219
93225
93241
93245
93259
93315
93326
93330
93333
93339
93343
93360
93361
93366
93374
93390
93416
93419
93437
93460
93470
93476
93485
93501
93531
93533
93544
93547
93571
93575
93592
93602
93609
93612
93619
93620
93624
93629
93665
93667
93670
93680
93702
93711
93718
93732
93752
93761
9376

...

139834
139860
139869
139885
139890
139928
139937
139948
139955
139960
139972
139981
139985
139994
139996
140031
140045
140055
140056
140064
140073
140076
140089
140094
140109
140110
140128
140163
140165
140171
140177
140212
140216
140226
140231
140252
140275
140277
140278
140281
140309
140314
140342
140363
140365
140400
140409
140417
140428
140441
140446
140484
140522
140536
140560
140579
140582
140586
140601
140615
140619
140623
140642
140666
140673
140682
140689
140700
140754
140767
140775
140788
140817
140827
140829
140832
140840
140848
140869
140874
140879
140892
140895
140917
140933
140957
140975
140981
140988
140997
141004
141016
141020
141064
141072
141084
141090
141100
141110
141117
141126
141136
141137
141143
141168
141169
141178
141186
141210
141229
141233
141244
141286
141293
141300
141311
141322
141330
141334
141346
141350
141356
141400
141429
141433
141483
141496
141506
141522
141533
141535
141554
141568
141573
141574
141587
141597
141599
141605
141607
141626
141632
141647

165381
165413
165416
165430
165442
165448
165449
165466
165469
165488
165493
165499
165503
165507
165524
165533
165537
165539
165555
165568
165581
165589
165595
165602
165615
165634
165678
165689
165712
165724
165746
165753
165765
165766
165769
165774
165786
165791
165797
165800
165807
165815
165816
165817
165824
165870
165875
165880
165883
165884
165928
165933
165955
165959
165968
165970
165976
165984
165985
165988
165996
166001
166016
166045
166051
166060
166062
166087
166088
166101
166103
166118
166133
166143
166158
166159
166180
166187
166190
166193
166209
166210
166243
166249
166254
166270
166320
166322
166327
166343
166349
166357
166381
166383
166389
166404
166411
166423
166430
166431
166469
166470
166477
166479
166486
166501
166502
166531
166547
166557
166558
166560
166565
166566
166567
166568
166578
166591
166602
166609
166616
166631
166634
166647
166648
166654
166663
166685
166688
166689
166705
166716
166721
166744
166746
166777
166783
166784
166791
166794
166809
166834
166837

181423
181429
181434
181440
181458
181467
181482
181489
181495
181497
181500
181517
181538
181581
181588
181596
181617
181631
181639
181647
181648
181653
181659
181672
181677
181703
181707
181714
181742
181754
181761
181769
181780
181790
181804
181805
181831
181845
181870
181877
181884
181889
181899
181902
181908
181918
181919
181925
181932
181951
181978
181979
181995
182130
182137
182163
182173
182192
182193
182195
182199
182205
182214
182237
182238
182249
182252
182253
182270
182278
182283
182287
182306
182307
182316
182320
182321
182330
182331
182336
182341
182343
182344
182352
182368
182369
182395
182403
182407
182464
182471
182472
182473
182489
182510
182514
182518
182545
182546
182548
182549
182573
182574
182577
182608
182611
182633
182635
182640
182662
182678
182680
182723
182757
182759
182768
182775
182797
182798
182810
182832
182835
182836
182873
182882
182888
182889
182896
182909
182934
182940
182942
182959
182964
182986
183038
183051
183054
183057
183073
183088
183100
183108

206968
206992
207003
207025
207032
207047
207060
207088
207108
207146
207147
207148
207156
207159
207165
207169
207186
207192
207197
207222
207225
207230
207233
207239
207246
207268
207284
207290
207323
207325
207335
207349
207353
207356
207369
207387
207389
207395
207402
207417
207418
207432
207460
207462
207469
207471
207479
207512
207566
207572
207575
207577
207581
207588
207589
207606
207612
207616
207627
207636
207644
207669
207674
207682
207707
207726
207740
207745
207752
207762
207765
207768
207771
207775
207805
207815
207816
207821
207823
207832
207854
207855
207878
207890
207891
207899
207901
207915
207930
207956
207971
207997
208004
208006
208013
208022
208028
208045
208074
208087
208088
208097
208099
208101
208109
208113
208118
208137
208151
208168
208169
208187
208188
208195
208203
208214
208217
208221
208236
208253
208261
208268
208280
208292
208326
208335
208342
208359
208367
208376
208406
208424
208428
208442
208449
208465
208475
208479
208497
208500
208552
208614
208628

229511
229512
229554
229560
229566
229569
229579
229589
229590
229616
229635
229648
229659
229678
229686
229692
229702
229707
229716
229725
229726
229733
229750
229752
229757
229758
229769
229775
229782
229784
229822
229825
229833
229835
229845
229858
229863
229879
229891
229898
229912
229917
229925
229927
229933
229964
229968
229974
229981
229985
230004
230007
230012
230013
230025
230045
230052
230067
230072
230077
230080
230116
230122
230131
230137
230149
230168
230169
230173
230177
230193
230210
230211
230237
230258
230289
230301
230323
230334
230358
230359
230410
230413
230425
230428
230431
230436
230446
230458
230472
230478
230487
230488
230491
230497
230520
230522
230563
230572
230577
230595
230617
230625
230631
230633
230637
230643
230655
230657
230677
230698
230717
230719
230731
230758
230759
230766
230769
230782
230790
230802
230821
230827
230828
230833
230837
230844
230853
230856
230886
230912
230923
230938
230950
230956
230965
230974
230995
231003
231013
231021
231034
231038

In [32]:
new_df_with_TSS_end

,Chromosome/scaffold name,TSS start,Strand,Gene stable ID,Transcript stable ID,Gene type,description,source,feature,score,frame,TSS end
5,MT,3307,+,ENSG00000198888,ENST00000361390,protein_coding,"Gene ID: ENSG00000198888, Transcript ID: ENST0...",lilach,TSS,1,.,3308.0
9,MT,4470,+,ENSG00000198763,ENST00000361453,protein_coding,"Gene ID: ENSG00000198763, Transcript ID: ENST0...",lilach,TSS,1,.,4471.0
15,MT,5904,+,ENSG00000198804,ENST00000361624,protein_coding,"Gene ID: ENSG00000198804, Transcript ID: ENST0...",lilach,TSS,1,.,5905.0
18,MT,7586,+,ENSG00000198712,ENST00000361739,protein_coding,"Gene ID: ENSG00000198712, Transcript ID: ENST0...",lilach,TSS,1,.,7587.0
20,MT,8366,+,ENSG00000228253,ENST00000361851,protein_coding,"Gene ID: ENSG00000228253, Transcript ID: ENST0...",lilach,TSS,1,.,8367.0
...,...,...,...,...,...,...,...,...,...,...,...,...
243276,9,61861625,+,ENSG00000204805,ENST00000605259,protein_coding,"Gene ID: ENSG00000204805, Transcript ID: ENST0...",lilach,TSS,1,.,61861626.0
243592,GL000009.2,58375,-,ENSG00000278704,ENST00000618686,protein_coding,"Gene ID: ENSG00000278704, Transcript ID: ENST0...",lilach,TSS,1,.,58376.0
243599,KI270728.1,1157687,+,ENSG00000278782,ENST00000619130,protein_coding,"Gene ID: ENSG00000278782, Transcript ID: ENST0...",lilach,TSS,1,.,1157688.0
243659,5,751233,-,ENSG00000286094,ENST00000622126,protein_coding,"Gene ID: ENSG00000286094, Transcript ID: ENST0...",lilach,TSS,1,.,751234.0


In [33]:
final_df=new_df_with_TSS_end[['Chromosome/scaffold name','source','feature','TSS start', 'TSS end','score','Strand','frame','description']]
final_df

,Chromosome/scaffold name,source,feature,TSS start,TSS end,score,Strand,frame,description
5,MT,lilach,TSS,3307,3308.0,1,+,.,"Gene ID: ENSG00000198888, Transcript ID: ENST0..."
9,MT,lilach,TSS,4470,4471.0,1,+,.,"Gene ID: ENSG00000198763, Transcript ID: ENST0..."
15,MT,lilach,TSS,5904,5905.0,1,+,.,"Gene ID: ENSG00000198804, Transcript ID: ENST0..."
18,MT,lilach,TSS,7586,7587.0,1,+,.,"Gene ID: ENSG00000198712, Transcript ID: ENST0..."
20,MT,lilach,TSS,8366,8367.0,1,+,.,"Gene ID: ENSG00000228253, Transcript ID: ENST0..."
...,...,...,...,...,...,...,...,...,...
243276,9,lilach,TSS,61861625,61861626.0,1,+,.,"Gene ID: ENSG00000204805, Transcript ID: ENST0..."
243592,GL000009.2,lilach,TSS,58375,58376.0,1,-,.,"Gene ID: ENSG00000278704, Transcript ID: ENST0..."
243599,KI270728.1,lilach,TSS,1157687,1157688.0,1,+,.,"Gene ID: ENSG00000278782, Transcript ID: ENST0..."
243659,5,lilach,TSS,751233,751234.0,1,-,.,"Gene ID: ENSG00000286094, Transcript ID: ENST0..."


In [34]:
# comparisons: 

final_df.loc[243592,:]


Chromosome/scaffold name                                           GL000009.2
source                                                                 lilach
feature                                                                   TSS
TSS start                                                               58375
TSS end                                                               58376.0
score                                                                       1
Strand                                                                      -
frame                                                                       .
description                 Gene ID: ENSG00000278704, Transcript ID: ENST0...
Name: 243592, dtype: object

In [35]:
new_df.loc[243592]

Chromosome/scaffold name                                           GL000009.2
TSS start                                                               58376
Strand                                                                      -
Gene stable ID                                                ENSG00000278704
Transcript stable ID                                          ENST00000618686
Gene type                                                      protein_coding
description                 Gene ID: ENSG00000278704, Transcript ID: ENST0...
source                                                                 lilach
feature                                                                   TSS
score                                                                       1
frame                                                                       .
Name: 243592, dtype: object

In [36]:
final_df

,Chromosome/scaffold name,source,feature,TSS start,TSS end,score,Strand,frame,description
5,MT,lilach,TSS,3307,3308.0,1,+,.,"Gene ID: ENSG00000198888, Transcript ID: ENST0..."
9,MT,lilach,TSS,4470,4471.0,1,+,.,"Gene ID: ENSG00000198763, Transcript ID: ENST0..."
15,MT,lilach,TSS,5904,5905.0,1,+,.,"Gene ID: ENSG00000198804, Transcript ID: ENST0..."
18,MT,lilach,TSS,7586,7587.0,1,+,.,"Gene ID: ENSG00000198712, Transcript ID: ENST0..."
20,MT,lilach,TSS,8366,8367.0,1,+,.,"Gene ID: ENSG00000228253, Transcript ID: ENST0..."
...,...,...,...,...,...,...,...,...,...
243276,9,lilach,TSS,61861625,61861626.0,1,+,.,"Gene ID: ENSG00000204805, Transcript ID: ENST0..."
243592,GL000009.2,lilach,TSS,58375,58376.0,1,-,.,"Gene ID: ENSG00000278704, Transcript ID: ENST0..."
243599,KI270728.1,lilach,TSS,1157687,1157688.0,1,+,.,"Gene ID: ENSG00000278782, Transcript ID: ENST0..."
243659,5,lilach,TSS,751233,751234.0,1,-,.,"Gene ID: ENSG00000286094, Transcript ID: ENST0..."


In [37]:
#biomart_protein_coding_no_CHR.sort_values(by='Strand', ascending=False)

# this happens in the original script
## in txt just in case, with headers and index 
final_df.to_csv(r'C:\Users\TzachiHNB1\Documents\docs_from_lilac\tzachi_hagai\stefan_bulk\promoter_analysis\homo\files_for_getfasta\gtf_TSS_human.txt',sep='\t')

## in gtf , no header and index
final_df.to_csv(r'C:\Users\TzachiHNB1\Documents\docs_from_lilac\tzachi_hagai\stefan_bulk\promoter_analysis\homo\files_for_getfasta\gtf_TSS_human.gtf',sep='\t',index=False,header=False)

In [38]:
final_df

,Chromosome/scaffold name,source,feature,TSS start,TSS end,score,Strand,frame,description
5,MT,lilach,TSS,3307,3308.0,1,+,.,"Gene ID: ENSG00000198888, Transcript ID: ENST0..."
9,MT,lilach,TSS,4470,4471.0,1,+,.,"Gene ID: ENSG00000198763, Transcript ID: ENST0..."
15,MT,lilach,TSS,5904,5905.0,1,+,.,"Gene ID: ENSG00000198804, Transcript ID: ENST0..."
18,MT,lilach,TSS,7586,7587.0,1,+,.,"Gene ID: ENSG00000198712, Transcript ID: ENST0..."
20,MT,lilach,TSS,8366,8367.0,1,+,.,"Gene ID: ENSG00000228253, Transcript ID: ENST0..."
...,...,...,...,...,...,...,...,...,...
243276,9,lilach,TSS,61861625,61861626.0,1,+,.,"Gene ID: ENSG00000204805, Transcript ID: ENST0..."
243592,GL000009.2,lilach,TSS,58375,58376.0,1,-,.,"Gene ID: ENSG00000278704, Transcript ID: ENST0..."
243599,KI270728.1,lilach,TSS,1157687,1157688.0,1,+,.,"Gene ID: ENSG00000278782, Transcript ID: ENST0..."
243659,5,lilach,TSS,751233,751234.0,1,-,.,"Gene ID: ENSG00000286094, Transcript ID: ENST0..."


# run from here if you want different promoters sizes, not nessasery to run from beggining

In [39]:
# creating the BED file, has only [chrom ] [ chromStart ] [chromEnd ]


In [40]:
for_BED=final_df.copy()
for_BED
final_df
for_BED

,Chromosome/scaffold name,source,feature,TSS start,TSS end,score,Strand,frame,description
5,MT,lilach,TSS,3307,3308.0,1,+,.,"Gene ID: ENSG00000198888, Transcript ID: ENST0..."
9,MT,lilach,TSS,4470,4471.0,1,+,.,"Gene ID: ENSG00000198763, Transcript ID: ENST0..."
15,MT,lilach,TSS,5904,5905.0,1,+,.,"Gene ID: ENSG00000198804, Transcript ID: ENST0..."
18,MT,lilach,TSS,7586,7587.0,1,+,.,"Gene ID: ENSG00000198712, Transcript ID: ENST0..."
20,MT,lilach,TSS,8366,8367.0,1,+,.,"Gene ID: ENSG00000228253, Transcript ID: ENST0..."
...,...,...,...,...,...,...,...,...,...
243276,9,lilach,TSS,61861625,61861626.0,1,+,.,"Gene ID: ENSG00000204805, Transcript ID: ENST0..."
243592,GL000009.2,lilach,TSS,58375,58376.0,1,-,.,"Gene ID: ENSG00000278704, Transcript ID: ENST0..."
243599,KI270728.1,lilach,TSS,1157687,1157688.0,1,+,.,"Gene ID: ENSG00000278782, Transcript ID: ENST0..."
243659,5,lilach,TSS,751233,751234.0,1,-,.,"Gene ID: ENSG00000286094, Transcript ID: ENST0..."


In [41]:
before_promoter=2000
after_promoter=500


In [42]:
for i in for_BED.index:
    strand=for_BED.at[i,'Strand']
    TSS_start=for_BED.at[i,'TSS start']
    

    
    if strand=='+':
        prom_end=round(TSS_start+after_promoter,0)
        prom_start=round(TSS_start-before_promoter,0)
        
        if prom_start<0:
            prom_start=0
            
        for_BED.at[i,'promoter_start']=prom_start
        for_BED.at[i,'promoter_end']=prom_end
        
        
    if strand=='-':
        prom_end_pre=round(TSS_start-after_promoter,0)
        prom_start_pre=round(TSS_start+before_promoter,0)
        
        prom_end=prom_start_pre
        prom_start=prom_end_pre
        
        if prom_start<0:
            prom_start=0
        
        for_BED.at[i,'promoter_start']=prom_start
        for_BED.at[i,'promoter_end']=prom_end

In [43]:
for_BED

,Chromosome/scaffold name,source,feature,TSS start,TSS end,score,Strand,frame,description,promoter_start,promoter_end
5,MT,lilach,TSS,3307,3308.0,1,+,.,"Gene ID: ENSG00000198888, Transcript ID: ENST0...",1307.0,3807.0
9,MT,lilach,TSS,4470,4471.0,1,+,.,"Gene ID: ENSG00000198763, Transcript ID: ENST0...",2470.0,4970.0
15,MT,lilach,TSS,5904,5905.0,1,+,.,"Gene ID: ENSG00000198804, Transcript ID: ENST0...",3904.0,6404.0
18,MT,lilach,TSS,7586,7587.0,1,+,.,"Gene ID: ENSG00000198712, Transcript ID: ENST0...",5586.0,8086.0
20,MT,lilach,TSS,8366,8367.0,1,+,.,"Gene ID: ENSG00000228253, Transcript ID: ENST0...",6366.0,8866.0
...,...,...,...,...,...,...,...,...,...,...,...
243276,9,lilach,TSS,61861625,61861626.0,1,+,.,"Gene ID: ENSG00000204805, Transcript ID: ENST0...",61859625.0,61862125.0
243592,GL000009.2,lilach,TSS,58375,58376.0,1,-,.,"Gene ID: ENSG00000278704, Transcript ID: ENST0...",57875.0,60375.0
243599,KI270728.1,lilach,TSS,1157687,1157688.0,1,+,.,"Gene ID: ENSG00000278782, Transcript ID: ENST0...",1155687.0,1158187.0
243659,5,lilach,TSS,751233,751234.0,1,-,.,"Gene ID: ENSG00000286094, Transcript ID: ENST0...",750733.0,753233.0


In [44]:
#final_df.loc[243592]

In [45]:
#new_df.loc[243592]

In [46]:
np.unique(for_BED['Chromosome/scaffold name'])

array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19',
       '2', '20', '21', '22', '3', '4', '5', '6', '7', '8', '9',
       'GL000009.2', 'GL000194.1', 'GL000195.1', 'GL000205.2',
       'GL000213.1', 'GL000218.1', 'GL000219.1', 'KI270711.1',
       'KI270713.1', 'KI270721.1', 'KI270726.1', 'KI270727.1',
       'KI270728.1', 'KI270731.1', 'KI270734.1', 'MT', 'X', 'Y'],
      dtype=object)

In [47]:
for_BED_no_scaffold=for_BED[(for_BED['Chromosome/scaffold name']=='MT')|
                           (for_BED['Chromosome/scaffold name']=='X')|
                           (for_BED['Chromosome/scaffold name']=='1')|
                           (for_BED['Chromosome/scaffold name']=='2')|
                           (for_BED['Chromosome/scaffold name']=='3')|
                           (for_BED['Chromosome/scaffold name']=='4')|
                           (for_BED['Chromosome/scaffold name']=='5')|
                           (for_BED['Chromosome/scaffold name']=='6')|
                           (for_BED['Chromosome/scaffold name']=='7')|
                           (for_BED['Chromosome/scaffold name']=='8')|
                           (for_BED['Chromosome/scaffold name']=='9')|
                           (for_BED['Chromosome/scaffold name']=='10')|
                           (for_BED['Chromosome/scaffold name']=='11')|
                           (for_BED['Chromosome/scaffold name']=='12')|
                           (for_BED['Chromosome/scaffold name']=='13')|
                           (for_BED['Chromosome/scaffold name']=='14')|
                           (for_BED['Chromosome/scaffold name']=='15')|
                           (for_BED['Chromosome/scaffold name']=='16')|
                           (for_BED['Chromosome/scaffold name']=='17')|
                           (for_BED['Chromosome/scaffold name']=='18')|
                           (for_BED['Chromosome/scaffold name']=='19')|
                           (for_BED['Chromosome/scaffold name']=='20')|
                           (for_BED['Chromosome/scaffold name']=='21')|
                           (for_BED['Chromosome/scaffold name']=='22')]

In [48]:
for_BED=for_BED_no_scaffold.copy()

In [49]:
final_BED=for_BED[['Chromosome/scaffold name','promoter_start','promoter_end']]

In [50]:
final_BED.rename(columns={'Chromosome/scaffold name':'chrom','promoter_start':'chromStart','promoter_end':'chromEnd'},inplace=True)
round(final_BED)
final_BED

C:\Users\TzachiHNB2\anaconda3\lib\site-packages\pandas\core\frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,chrom,chromStart,chromEnd
5,MT,1307.0,3807.0
9,MT,2470.0,4970.0
15,MT,3904.0,6404.0
18,MT,5586.0,8086.0
20,MT,6366.0,8866.0
...,...,...,...
237865,9,67716236.0,67718736.0
238227,9,113049542.0,113052042.0
240924,9,61862066.0,61864566.0
243276,9,61859625.0,61862125.0


In [51]:
final_BED=pd.concat([final_BED,new_df_with_TSS_end['Gene stable ID']],axis=1)
final_BED.columns
final_BED=final_BED.dropna()
final_BED

,chrom,chromStart,chromEnd,Gene stable ID
5,MT,1307.0,3807.0,ENSG00000198888
9,MT,2470.0,4970.0,ENSG00000198763
15,MT,3904.0,6404.0,ENSG00000198804
18,MT,5586.0,8086.0,ENSG00000198712
20,MT,6366.0,8866.0,ENSG00000228253
...,...,...,...,...
237865,9,67716236.0,67718736.0,ENSG00000286079
238227,9,113049542.0,113052042.0,ENSG00000285447
240924,9,61862066.0,61864566.0,ENSG00000170152
243276,9,61859625.0,61862125.0,ENSG00000204805


In [52]:
final_BED=final_BED.sort_values(by='chrom')
final_BED

,chrom,chromStart,chromEnd,Gene stable ID
70326,1,84892705.0,84895205.0,ENSG00000171517
43884,1,15734258.0,15736758.0,ENSG00000162461
43897,1,27671697.0,27674197.0,ENSG00000126709
147696,1,196941738.0,196944238.0,ENSG00000080910
221164,1,151059121.0,151061621.0,ENSG00000197622
...,...,...,...,...
179736,X,101548547.0,101551047.0,ENSG00000126947
225826,X,20266408.0,20268908.0,ENSG00000177189
146102,X,34940796.0,34943296.0,ENSG00000189132
188668,X,153685926.0,153688426.0,ENSG00000130821


In [53]:
final_BED.rename(columns={'Gene stable ID':'name'},inplace=True)
final_BED

,chrom,chromStart,chromEnd,name
70326,1,84892705.0,84895205.0,ENSG00000171517
43884,1,15734258.0,15736758.0,ENSG00000162461
43897,1,27671697.0,27674197.0,ENSG00000126709
147696,1,196941738.0,196944238.0,ENSG00000080910
221164,1,151059121.0,151061621.0,ENSG00000197622
...,...,...,...,...
179736,X,101548547.0,101551047.0,ENSG00000126947
225826,X,20266408.0,20268908.0,ENSG00000177189
146102,X,34940796.0,34943296.0,ENSG00000189132
188668,X,153685926.0,153688426.0,ENSG00000130821


In [54]:
convert_dict = {'chromStart': int,
                'chromEnd': int }  
  
final_BED = final_BED.astype(convert_dict)  
print(final_BED.dtypes)

chrom         object
chromStart     int32
chromEnd       int32
name          object
dtype: object


In [55]:
final_BED

,chrom,chromStart,chromEnd,name
70326,1,84892705,84895205,ENSG00000171517
43884,1,15734258,15736758,ENSG00000162461
43897,1,27671697,27674197,ENSG00000126709
147696,1,196941738,196944238,ENSG00000080910
221164,1,151059121,151061621,ENSG00000197622
...,...,...,...,...
179736,X,101548547,101551047,ENSG00000126947
225826,X,20266408,20268908,ENSG00000177189
146102,X,34940796,34943296,ENSG00000189132
188668,X,153685926,153688426,ENSG00000130821


In [82]:
# in BED , no header and index
# not relevant : 
#final_BED.to_csv(r'C:\Users\TzachiHNB1\Documents\docs_from_lilac\tzachi_hagai\my_project_from_22_11\fimo\bed_files_all_genes\BED_promoters_human_bef{}_aft{}_no_headers_chr_not_in_chrname.bed'.format(before_promoter,after_promoter),sep='\t',index=False,header=False)


In [56]:
final_BED_rep=final_BED.copy()
final_BED_rep

,chrom,chromStart,chromEnd,name
70326,1,84892705,84895205,ENSG00000171517
43884,1,15734258,15736758,ENSG00000162461
43897,1,27671697,27674197,ENSG00000126709
147696,1,196941738,196944238,ENSG00000080910
221164,1,151059121,151061621,ENSG00000197622
...,...,...,...,...
179736,X,101548547,101551047,ENSG00000126947
225826,X,20266408,20268908,ENSG00000177189
146102,X,34940796,34943296,ENSG00000189132
188668,X,153685926,153688426,ENSG00000130821


In [57]:
chroms_names_dict={i:'chr{}'.format(i) for i in (np.unique(final_BED_rep['chrom'])) if i!='MT'}
final_BED_rep.replace(chroms_names_dict,inplace=True)
final_BED_rep

,chrom,chromStart,chromEnd,name
70326,chr1,84892705,84895205,ENSG00000171517
43884,chr1,15734258,15736758,ENSG00000162461
43897,chr1,27671697,27674197,ENSG00000126709
147696,chr1,196941738,196944238,ENSG00000080910
221164,chr1,151059121,151061621,ENSG00000197622
...,...,...,...,...
179736,chrX,101548547,101551047,ENSG00000126947
225826,chrX,20266408,20268908,ENSG00000177189
146102,chrX,34940796,34943296,ENSG00000189132
188668,chrX,153685926,153688426,ENSG00000130821


In [85]:
final_BED=final_BED_rep.copy()

In [86]:
# in txt just in case, with headers and index 
# old path :final_BED.to_csv(r'C:\Users\TzachiHNB1\Documents\docs_from_lilac\tzachi_hagai\my_project_from_22_11\more_oneTOone_analysis\chip_seq\creating_promoters_files\bed_files_promoters\BED_promoters_human_bef{}_aft{}.txt'.format(before_promoter,after_promoter),sep='\t')
final_BED.to_csv(r'bed_files_promoters\BED_promoters_human_bef{}_aft{}.txt'.format(before_promoter,after_promoter),sep='\t')

In [87]:
# in BED , no header and index
# old path : final_BED.to_csv(r'C:\Users\TzachiHNB1\Documents\docs_from_lilac\tzachi_hagai\my_project_from_22_11\more_oneTOone_analysis\chip_seq\creating_promoters_files\bed_files_promoters\BED_promoters_human_bef{}_aft{}_no_headers.bed'.format(before_promoter,after_promoter),sep='\t',index=False,header=False)

final_BED.to_csv(r'bed_files_promoters\BED_promoters_human_bef{}_aft{}_no_headers.bed'.format(before_promoter,after_promoter),sep='\t',index=False,header=False)


In [88]:
# in BED , no index, yes header
final_BED.to_csv(r'bed_files_promoters\BED_promoters_human_bef{}_aft{}.bed'.format(before_promoter,after_promoter),sep='\t',index=False)